In [1]:
import tensorflow as tf
import tensorlayer as tl
import numpy as np

In [2]:
import os
from sklearn.model_selection import train_test_split

In [3]:
# Helper function to load images information (image_path, image_label)
def load_image_path(data_path, valid_proportion, test_proportion):
    pos_image_path = []
    pos_labels = []

    neg_image_path = []
    neg_labels = []

    ful_image_path = []
    ful_labels = []

    np.random.seed(0)

    pos_path = data_path + "1/"
    for img in tf.gfile.ListDirectory(pos_path):
        # if '.bmp' not in img:
        #    continue

        label = 1

        path = os.path.join(pos_path, img)
        pos_image_path.append(path)
        pos_labels.append(label)

    neg_path = data_path + "0/"
    for img in tf.gfile.ListDirectory(neg_path):
        # if '.bmp' not in img:
        #    continue

        label = 0

        path = os.path.join(neg_path, img)
        neg_image_path.append(path)
        neg_labels.append(label)

    ful_image_path = pos_image_path + neg_image_path
    ful_labels = pos_labels + neg_labels

    temp = np.array([ful_image_path, ful_labels])
    temp = temp.transpose()
    np.random.shuffle(temp)
    ful_image_path = list(temp[:, 0])
    ful_labels = list(temp[:, 1])
    ful_labels = [int(i) for i in ful_labels]

    x_train, x_valid, y_train, y_valid = train_test_split(ful_image_path, ful_labels,
                                                          test_size=(valid_proportion + test_proportion),
                                                          stratify=ful_labels, random_state=1)
    x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=test_proportion / (
            valid_proportion + test_proportion), stratify=y_valid, random_state=1)

    print("train_num: %d ,pos_num: %d , neg_num: %d" % (
        len(y_train), count_pos(y_train), len(y_train) - count_pos(y_train)))
    print("valid_num: %d ,pos_num: %d , neg_num: %d" % (
        len(y_valid), count_pos(y_valid), len(y_valid) - count_pos(y_valid)))
    print("test_num : %d ,pos_num: %d , neg_num: %d" % (
        len(y_test), count_pos(y_test), len(y_test) - count_pos(y_test)))

    return x_train, y_train, x_valid, y_valid, x_test, y_test


def count_pos(lables):
    num = 0
    for i in range(len(lables)):
        if lables[i] == 1:
            num = num + 1
    return num

In [4]:
Train_Proportion = 0.8  # Proportion of the data to be used for training
Valid_Proportion = 0.1
Test_Proportion = 0.1  #
data_dir = "F:/Github/clone/image_food_classification/datasets/food_101_dataset/"

In [5]:
x_train, y_train, x_valid, y_valid, x_test, y_test = load_image_path(data_dir, Valid_Proportion,Test_Proportion)

train_num: 3593 ,pos_num: 757 , neg_num: 2836
valid_num: 449 ,pos_num: 95 , neg_num: 354
test_num : 450 ,pos_num: 95 , neg_num: 355


In [ ]:
'''
上面是图片路径和标签的获取，在另外的例程里有详细介绍
'''

In [7]:
'''
输入的格式，x是图片的绝对路径，字符串组成的列表
            y是标签值，0或者1，int组成的列表
'''
x = x_test[0:10]
y = y_test[0:10]
print(x[:2])
print(y)

['F:/Github/clone/image_food_classification/datasets/food_101_dataset/0/club_sandwich_65.jpg', 'F:/Github/clone/image_food_classification/datasets/food_101_dataset/0/club_sandwich_767.jpg']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
def get_batch(x, y, image_W, image_H, batch_size, capacity=500, epochs = None):
    '''
    Args:
        x: list type
        y: list type
        image_W: image width
        image_H: image height
        batch_size: batch size
        capacity: the maximum elements in queue
        epochs: 如果要指定epoch数
    Returns:
        image_batch: 4D tensor [batch_size, width, height, 3], dtype=tf.float32
        label_batch: 1D tensor [batch_size], dtype=tf.int32
    '''

    x = tf.cast(x, tf.string)
    y = tf.cast(y, tf.int32)

    # make an input queue
    input_queue = tf.train.slice_input_producer([x, y],num_epochs=epochs)

    label = input_queue[1]
    image_contents = tf.read_file(input_queue[0])
    image = tf.image.decode_jpeg(image_contents, channels=3)

    ######################################
    # data argumentation should go to here
    ######################################

    image = tf.image.resize_images(image, [image_W, image_H], method=0)
    # image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H)
    # if you want to test the generated batches of images, you might want to comment the following line.
    # 如果想看到正常的图片，请注释掉111行（标准化）和 126行（image_batch = tf.cast(image_batch, tf.float32)）
    # 训练时不要注释掉！
    image = tf.image.per_image_standardization(image)

    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,
                                              capacity=capacity)

    image_batch = tf.cast(image_batch, tf.float32)
    label_batch = tf.reshape(label_batch, [batch_size])

    # label_batch = to_categorical(y=label_batch, nb_classes=2)
    return image_batch, label_batch

In [9]:
'''
如果我指定获取batch的epoch是1，batch_size是3，那么10条数据只获得3个batch，在尝试获取第四个batch的时候就超出范围了就抛出异常了。
'''
test_batch_run, test_label_batch_run = get_batch(x,y,208,208,batch_size=3,epochs=1)
with tf.Session() as sess:
    
    tf.local_variables_initializer().run()#虽然好像没有用到变量，但是用到tf.train.batch就要加这句话
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
   
    try:
        while not coord.should_stop() :
           
            img, label = sess.run([test_batch_run, test_label_batch_run])
            print(label)
           
    except tf.errors.OutOfRangeError:
        print('done!')
    finally:
        coord.request_stop()
    coord.join(threads)

[0 0 0]
[0 0 0]
[0 0 0]
done!


In [10]:
'''
如果我不指定获取batch的epoch，只指定batch_size是3，那么10条数据就会不断循环，不断获取size是3的batch。这里我是用i来限定5次循环读取batch
'''
test_batch_run, test_label_batch_run = get_batch(x,y,208,208,batch_size=3)
with tf.Session() as sess:
    
    tf.local_variables_initializer().run()#虽然好像没有用到变量，但是用到tf.train.batch就要加这句话
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    i = 0
    try:
        while not coord.should_stop() and i<5:
           
            img, label = sess.run([test_batch_run, test_label_batch_run])
            print(label)
            i += 1 
    except tf.errors.OutOfRangeError:
        print('done!')
    finally:
        coord.request_stop()
    coord.join(threads)

[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
